In [7]:

import os, sys
from cellpose import utils, io, models
from tqdm import tqdm
import numpy as np

days = ["0day", "3day", "5day", "7day", "11day", "14day"]
flow_thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

#days = ["11day", "14day"]
channels = [[3,0]] #0 for gray, 1 for red, 2 for green, 3 for blue
diameter = 30

resample =  True
use_gpu = True

model_paths = ["../tuning_model/CP_20231201_111120_Ours4", "../tuning_model/GroundTrueModel_ver3"]

for model_path in model_paths:

    for flow_threshold in flow_thresholds:

        #model_path = "Cellpose_retrained_model/CP_20220420_140301"
        #model_path =  "../tuning_model/CP_20231201_111120_Ours4"
        #model_path =  "../tuning_model/GroundTrueModel_ver3"

        input_path = "/a/yu-yamaoka/data/valid_rawdata_256_tiff/"

        output_folder = os.path.join("EXPERIENCE1", os.path.basename(model_path) + "_" + str(flow_threshold))
        if(os.path.exists(output_folder)==False):
            os.mkdir(output_folder)

        #日付ごとに実行
        for day in days:
            #日付ごとのフォルダ作成
            in_path =  os.path.join(input_path, day, "image") 
            out_path = os.path.join(output_folder, day)
            if(os.path.exists(out_path)==False):
                os.mkdir(out_path)

            # Loads model
            model = models.CellposeModel(gpu=use_gpu, pretrained_model=model_path)   

            #For each tile of image to be segmented
            for tile in tqdm(os.listdir(in_path)):
                #print("Cellpose will start for image ", format(tile))
                img = io.imread(in_path + "/" + tile)

                # Computes segmentations
                masks, flows, styles = model.eval(img, diameter=diameter, flow_threshold=flow_threshold, resample=resample, channels=channels)

                # Save results as png
                io.save_masks(img, masks, flows, tile, savedir=out_path)
                np.save(os.path.join(out_path, tile),  masks)

100%|██████████| 35/35 [00:43<00:00,  1.23s/it]


In [7]:

import os, sys
from cellpose import utils, io, models
from tqdm import tqdm
import numpy as np

days = ["0day", "3day", "5day", "7day", "11day", "14day"]
#days = ["11day", "14day"]

for day in days:
    in_path = "/a/yu-yamaoka/data/valid_rawdata_256_tiff/" + day + "/image"
    out_path = "./EXPERIENCE1/Valid_output_MyoSothes/" + day
    use_gpu = True
    
    #model_path =  "../tuning_model/CP_20231201_111120_Ours4"#/ #"" #."../tuning_model/yamaoka0428_3" #or model name for default models
    model_path = "Cellpose_retrained_model/CP_20220420_140301"
    
    channels = [[3,0]] #0 for gray, 1 for red, 2 for green, 3 for blue
    diameter = 30
    flow_threshold = 0.5
    resample =  True

    if(os.path.exists(out_path)==False):
        os.mkdir(out_path)

    # Loads model
    model = models.CellposeModel(gpu=use_gpu, pretrained_model=model_path)   

    #For each tile of image to be segmented
    for tile in tqdm(os.listdir(in_path)):
        #print("Cellpose will start for image ", format(tile))
        img = io.imread(in_path + "/" + tile)

        # Computes segmentations
        masks, flows, styles = model.eval(img, diameter=diameter, flow_threshold=flow_threshold, resample=resample, channels=channels)

        # Save results as png
        io.save_masks(img, masks, flows, tile, savedir=out_path)
        np.save(os.path.join(out_path, tile),  masks)

100%|██████████| 165/165 [02:01<00:00,  1.35it/s]
